In [1]:
import warnings
import textwrap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy
from matplotlib import pyplot, transforms
from matplotlib.text import OffsetFrom

In [2]:
df = pd.read_csv('./data/kaggle_survey_2022_responses.csv', skiprows=[0])

/var/folders/4c/6jy3z6sn7zg_n_24fwj6v_xw0000gn/T/ipykernel_80538/2989873918.py:1: DtypeWarning: Columns (208,225,255,257,260,270,271,277) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/kaggle_survey_2022_responses.csv', skiprows=[0])


# Data cleaning

## Columns

In [3]:
col_mapping = pd.read_csv('./data/column_mapping.csv')

In [4]:
col_mapping_dict = {k:v for k, v in col_mapping.dropna().values}

In [5]:
df = df[col_mapping_dict.keys()].rename(columns=col_mapping_dict)

## Select data professionalas only


In [6]:
pos_mapping = pd.read_csv('./data/position_mapping.csv')
pos_maping_dict = {k:v for k, v in pos_mapping.dropna().values}

In [7]:
df['title'] = df['title'].map(pos_maping_dict)

In [8]:
df = df.query('student == "No"')

In [9]:
#Drop the column student after filtering to focus on industry insights from professionals
df = df.drop(columns=['student'])
df.head(3)

,age,gender,country,education_level,published,ml_used_theoretical,ml_used_applied,ml_used_no,code_exp,program_lang_Python,...,cloud_platform_GoogleCloudPlatform(GCP),cloud_platform_IBMCloud/RedHat,cloud_platform_OracleCloud,cloud_platform_SAPCloud,cloud_platform_VMwareCloud,cloud_platform_AlibabaCloud,cloud_platform_TencentCloud,cloud_platform_HuaweiCloud,cloud_platform_None,cloud_platform_Other
0,30-34,Man,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30-34,Man,Algeria,Master’s degree,Yes,NaN,NaN,No,1-3 years,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55-59,Man,France,Some college/university study without earning ...,NaN,NaN,NaN,NaN,10-20 years,Python,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other


In [10]:
df = df.query('title.notna() and title != "Currently not employed"')


In [11]:
df['title'].unique()

array(['Data Scientist', 'Software Engineer', 'Research Scientist',
       'Other', 'Developer Advocate', 'Data Analyst', 'Data Engineer',
       'ML/MLops Engineer', 'Engineer (non-software)',
       'Teacher/professor', 'Statistician', 'Manager',
       'Data Administrator', 'Data Architect'], dtype=object)

In [12]:
df = df.query('industry.notna()')
df.head(3)

,age,gender,country,education_level,published,ml_used_theoretical,ml_used_applied,ml_used_no,code_exp,program_lang_Python,...,cloud_platform_GoogleCloudPlatform(GCP),cloud_platform_IBMCloud/RedHat,cloud_platform_OracleCloud,cloud_platform_SAPCloud,cloud_platform_VMwareCloud,cloud_platform_AlibabaCloud,cloud_platform_TencentCloud,cloud_platform_HuaweiCloud,cloud_platform_None,cloud_platform_Other
3,55-59,Man,France,Some college/university study without earning ...,NaN,NaN,NaN,NaN,10-20 years,Python,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
7,30-34,Man,Germany,Bachelor’s degree,NaN,NaN,NaN,NaN,10-20 years,Python,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,70+,Man,Australia,Doctoral degree,Yes,NaN,"Yes, the research made use of machine learning...",NaN,20+ years,Python,...,Google Cloud Platform (GCP),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#Plotting the proportions of country 
num_country = df['country'].value_counts()
num_country = num_country.reset_index()
num_country

,index,country
0,India,2678
1,United States of America,1586
2,Other,579
3,Japan,370
4,Brazil,315
5,Nigeria,168
6,France,160
7,United Kingdom of Great Britain and Northern I...,150
8,Spain,150
9,Mexico,149


Encode Machine learning stages

In [14]:
df['company_stage'].unique()

array(['We recently started using ML methods (i.e., models in production for less than 2 years)',
       'We have well established ML methods (i.e., models in production for more than 2 years)',
       'We are exploring ML methods (and may one day put a model into production)',
       'I do not know',
       'We use ML methods for generating insights (but do not put working models into production)',
       'No (we do not use ML methods)', nan], dtype=object)

In [15]:
company_stage_dict = {
    'I do not know': -1,
    'No (we do not use ML methods)': 0,
    'We are exploring ML methods (and may one day put a model into production)': 1,
    'We use ML methods for generating insights (but do not put working models into production)': 2,
    'We recently started using ML methods (i.e., models in production for less than 2 years)': 3,
    'We have well established ML methods (i.e., models in production for more than 2 years)': 4
}



In [16]:
df['stage_index'] = df['company_stage'].map(company_stage_dict)

In [17]:
df.to_csv('./data/cleaned_kaggle2022.csv', index=False)

## 1. Gender & Title distribution in different industries

In [18]:
position = df['title'].value_counts()
position = position.reset_index()
position

,index,title
0,Data Scientist,1913
1,Data Analyst,1513
2,Software Engineer,971
3,Manager,826
4,Teacher/professor,820
5,Other,748
6,Research Scientist,590
7,ML/MLops Engineer,562
8,Engineer (non-software),463
9,Data Engineer,341


In [19]:
position['Group'] = position['index'].where(position['title'] > 100, 'Other')
position

,index,title,Group
0,Data Scientist,1913,Data Scientist
1,Data Analyst,1513,Data Analyst
2,Software Engineer,971,Software Engineer
3,Manager,826,Manager
4,Teacher/professor,820,Teacher/professor
5,Other,748,Other
6,Research Scientist,590,Research Scientist
7,ML/MLops Engineer,562,ML/MLops Engineer
8,Engineer (non-software),463,Engineer (non-software)
9,Data Engineer,341,Data Engineer


In [20]:
group_position = position.groupby('Group')['title'].sum().reset_index().sort_values(by='title', ascending=False)
group_position

,Group,title
2,Data Scientist,1913
0,Data Analyst,1513
6,Other,972
8,Software Engineer,971
5,Manager,826
10,Teacher/professor,820
7,Research Scientist,590
4,ML/MLops Engineer,562
3,Engineer (non-software),463
1,Data Engineer,341


In [21]:
def plot_positions(group_position, ax):
    labels = group_position['Group']
    sizes = group_position['title']
    colors = ['#A6ABAD','#00587A', '#0073A1', '#00A1E0','#00BCE3','#87CEEB', '#89BCC4', '#9BD3DD', '#A4E0EB']
    
    patches, labels_, percentages = ax.pie(
        sizes, colors=colors,
        wedgeprops={'linewidth': 3.0, 'edgecolor': 'white'},
        textprops={'color': 'white', 'fontweight': 'bold','fontname': "Sans Serif"},
        startangle=90, frame=True,
        autopct="%.2f%%",
        pctdistance=0.85,
    )

    ax.axis('off')

    ax.add_artist(plt.Circle((0, 0), 0.6, color='white', linewidth=0))

    # Adding Title of chart
    ax.set_title('Popular positions from surveyors', fontweight = 'bold', size = 15, fontfamily='serif', ha="center", color="#4d4d4d")

    ax.legend(labels, loc='upper right', bbox_to_anchor=(1.35, 0.75))


In [22]:
# fig, axs = plt.subplots(figsize=(15,9), dpi=400)
# plot_positions(group_position, axs)
# plt.show()

In [23]:
df['count'] = 1

In [24]:
position_order = sorted(df['title'].unique().tolist())
position_order

['Data Administrator',
 'Data Analyst',
 'Data Architect',
 'Data Engineer',
 'Data Scientist',
 'Developer Advocate',
 'Engineer (non-software)',
 'ML/MLops Engineer',
 'Manager',
 'Other',
 'Research Scientist',
 'Software Engineer',
 'Statistician',
 'Teacher/professor']

In [25]:
industry_order = sorted(df['industry'].unique().tolist())
industry_order

['Academics/Education',
 'Accounting/Finance',
 'Broadcasting/Communications',
 'Computers/Technology',
 'Energy/Mining',
 'Government/Public Service',
 'Insurance/Risk Assessment',
 'Manufacturing/Fabrication',
 'Marketing/CRM',
 'Medical/Pharmaceutical',
 'Non-profit/Service',
 'Online Service/Internet-based Services',
 'Other',
 'Retail/Sales',
 'Shipping/Transportation']

In [26]:
data_q5q15 = pd.pivot_table(df, values='count', index=['title'], columns=['industry'], aggfunc=np.sum).fillna(0).astype(int).loc[position_order, industry_order].stack()
data_q5q15_man = pd.pivot_table(df[df['gender']=='Man'], values='count', index=['title'], columns=['industry'], aggfunc=np.sum).fillna(0).astype(int).loc[position_order, industry_order].stack()
data_q5q15_woman = pd.pivot_table(df[df['gender']=='Woman'], values='count', index=['title'], columns=['industry'], aggfunc=np.sum).fillna(0).astype(int).loc[position_order, industry_order].stack()

In [27]:
def drawPieMarker(xs, ys, ratios, sizes, colors, ax):
    markers = []
    previous = 0
    # calculate the points of the pie pieces
    for color, ratio in zip(colors, ratios):
        this = 2 * np.pi * ratio + previous
        x  = [0] + np.cos(np.linspace(previous, this, 30)).tolist() + [0]
        y  = [0] + np.sin(np.linspace(previous, this, 30)).tolist() + [0]
        xy = np.column_stack([x, y])
        previous = this
        markers.append({'marker':xy, 's':np.abs(xy).max()**2*np.array(sizes), 'facecolor':color})

    # scatter each of the pie pieces to create pies
    for marker in markers:
        ax.scatter(xs, ys, **marker, alpha=0.7)

In [28]:
# Define function for marker
# def count_to_size(val):
#     max_sz = 20
#     min_sz = 3
#     return np.sqrt((val - 1) / (80 - 1)) * (max_sz - min_sz) + min_sz

In [29]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    fig = plt.figure(figsize=(22, 25), dpi=200)

    gs = fig.add_gridspec(5, 5)

    ax_plot = fig.add_subplot(gs[1:4, 0:4]) 
    for q5_idx in position_order[::-1]:
        for q15_idx in industry_order:
            man = data_q5q15_man[q5_idx][q15_idx]
            woman = data_q5q15_woman[q5_idx][q15_idx]
            tot = data_q5q15[q5_idx][q15_idx]
            drawPieMarker([q15_idx],[q5_idx], [man/(man+woman), woman/(man+woman)], [tot*7], ['#1170aa', '#B21807'], ax=ax_plot)

    ax_plot.grid(linewidth=0.2, zorder=0)        

    ax_plot.tick_params(axis='x', labelrotation=90)

    # region Industry
    ax_int = fig.add_subplot(gs[0, :4], sharex=ax_plot) 
    data_q15_woman = df[df['gender']=='Woman']['industry'].value_counts()[industry_order]
    ax_int.bar(data_q15_woman.index, data_q15_woman, width=0.45, alpha=0.7, color='#B21807', label='Female')

    data_q15_man = df[df['gender']=='Man']['industry'].value_counts()[industry_order]
    ax_int.bar(data_q15_man.index, data_q15_man, bottom=data_q15_woman , width=0.45, alpha=0.7, color='#1170aa', label='Male')

    plt.setp(ax_int.get_xticklabels(), visible=False)
    # endregion


    # region Title
    ax_tit = fig.add_subplot(gs[1:4, 4], sharey=ax_plot) 

    data_q5_woman = df[df['gender']=='Woman']['title'].value_counts()[position_order]
    ax_tit.barh(data_q5_woman.index[::-1], data_q5_woman[::-1], height=0.55, alpha=0.7, color='#B21807')

    data_q5_man = df[df['gender']=='Man']['title'].value_counts()[position_order]
    ax_tit.barh(data_q5_man.index[::-1], data_q5_man[::-1], left= data_q5_woman[::-1],height=0.55, alpha=0.7, color='#1170aa')

    plt.setp(ax_tit.get_yticklabels(), visible=False)
    # endregion

    # Spines
    for s in ['top', 'left', 'right', 'bottom']:
        ax_plot.spines[s].set_visible(False)
        ax_int.spines[s].set_visible(False)
        ax_tit.spines[s].set_visible(False)
        

    fig.text(0.6, 0.9, 'Gender & Title distribution by Industry', fontweight='bold', fontfamily='arial', fontsize=35, ha='right', color = '#C41E3A') 
    fig.text(0.6, 0.88, 'Source: Data Professionals - Kaggle Survey 2022', fontweight='light', style= 'italic', fontfamily='arial', fontsize=15, ha='right')

    # Legend
    legend_gender = ax_int.legend(bbox_to_anchor=(1.2, 1.1), fontsize=16, frameon=False, title='Gender', title_fontsize=20)
    legend_count = ax_int.annotate(
        'Count of professionals',
        xy=(0.1, 0.1),
        textcoords=OffsetFrom(legend_gender, (0.5, -0.5)),
        xytext=(0, 0),
        fontsize=20,
        ha='center',
    )

    corner_ax = fig.add_subplot(gs[0, 4], zorder=-1)
    corner_ax.set_axis_off()
    for s in ['top', 'left', 'right', 'bottom']:
        corner_ax.spines[s].set_visible(False)

    ax_count = corner_ax.inset_axes([0, 0, 1, 0.5])
    ax_count.set_axis_off()
    sizes = [50, 100, 200]
    x_coords = [0] * len(sizes)
    y_coords = np.arange(len(sizes))
    ax_count.scatter(x_coords, y_coords, marker='o', s=[size*7 for size in sizes], c='#a3acb9', alpha=0.7)
    ax_count.set_xlim(-0.02, 0.06)
    ax_count.set_ylim(-0.5, 2.5)

    for size, x, y in zip(sizes, x_coords, y_coords):
        ax_count.annotate(str(size), (x + 0.015, y-0.15), fontsize=16)

    plt.savefig('./charts/gender title by industry.png')
    plt.close('all')

## 2. ML in research by data professionals

In [30]:
df = df.query('education_level.notna() and education_level != "I prefer not to answer"')
df['education_level'].unique()

array(['Some college/university study without earning a bachelor’s degree',
       'Bachelor’s degree', 'Doctoral degree', 'Master’s degree',
       'No formal education past high school', 'Professional doctorate'],
      dtype=object)

In [31]:
df['edu_group'] = df['education_level'].where(
    ~df['education_level'].isin(["No formal education past high school", "Some college/university study without earning a bachelor’s degree"]), "Below Bachelor's Degree"
    )

In [32]:
df['edu_group'] = df['edu_group'].where(
    ~df['edu_group'].isin(["Professional doctorate", "Doctoral degree"]), "Above Master's Degree"
    )

In [33]:
df['edu_group'].unique()

array(["Below Bachelor's Degree", 'Bachelor’s degree',
       "Above Master's Degree", 'Master’s degree'], dtype=object)

In [34]:
df_theo = df.query('ml_used_theoretical.notna()').groupby(by='title').count()
df_theo

,age,gender,country,education_level,published,ml_used_theoretical,ml_used_applied,ml_used_no,code_exp,program_lang_Python,...,cloud_platform_SAPCloud,cloud_platform_VMwareCloud,cloud_platform_AlibabaCloud,cloud_platform_TencentCloud,cloud_platform_HuaweiCloud,cloud_platform_None,cloud_platform_Other,stage_index,count,edu_group
title,,,,,,,,,,,,,,,,,,,,,
Data Administrator,3,3,3,3,3,3,1,0,3,2,...,0,0,0,0,0,0,0,3,3,3
Data Analyst,45,45,45,45,45,45,18,0,45,39,...,2,4,2,2,1,8,1,43,45,45
Data Architect,5,5,5,5,5,5,3,0,5,5,...,0,0,0,0,0,1,0,5,5,5
Data Engineer,14,14,14,14,14,14,5,0,14,14,...,1,2,2,1,2,1,0,14,14,14
Data Scientist,185,185,185,185,185,185,95,0,185,178,...,4,3,2,2,0,13,4,180,185,185
Developer Advocate,7,7,7,7,7,7,0,0,7,3,...,1,1,1,1,1,1,0,7,7,7
Engineer (non-software),7,7,7,7,7,7,3,0,7,6,...,0,0,0,0,0,0,0,7,7,7
ML/MLops Engineer,70,70,70,70,70,70,34,0,70,68,...,1,2,2,1,2,1,5,70,70,70
Manager,49,49,49,49,49,49,26,0,49,38,...,0,3,0,0,0,8,3,48,49,49


In [35]:
from pandas.plotting import parallel_coordinates

In [36]:
ml_used_df = df.dropna(subset=['published'], how='all')
title_count = ml_used_df.groupby('title').size()
ml_theoretical = ml_used_df.groupby('title')['ml_used_theoretical'].count()
ml_applied = ml_used_df.groupby('title')['ml_used_applied'].count()
ml_no = ml_used_df.groupby('title')['ml_used_no'].count()
ml_used = pd.concat([title_count, ml_theoretical, ml_applied, ml_no], axis=1)
ml_used = ml_used.rename(columns={0: 'count'}).reset_index()
ml_used = (
    ml_used
    .eval('pct_theoretical = ml_used_theoretical / count * 100')
    .eval('pct_applied = ml_used_applied / count * 100')
    .eval('pct_no= ml_used_no / count * 100')
    .drop(['ml_used_theoretical', 'ml_used_applied', 'ml_used_no', 'count'], axis=1)
    .sort_values('pct_theoretical')
    .reset_index(drop=True)
)
ml_used

,title,pct_theoretical,pct_applied,pct_no
0,Engineer (non-software),2.941176,12.184874,30.672269
1,Other,4.615385,9.230769,25.384615
2,Data Analyst,5.617978,10.362047,17.602996
3,Software Engineer,8.008658,15.367965,17.532468
4,Data Engineer,8.187135,15.204678,15.204678
5,Manager,8.626761,17.957746,17.957746
6,Data Architect,9.433962,13.207547,13.207547
7,Data Administrator,9.677419,6.451613,9.677419
8,Statistician,12.222222,22.222222,18.888889
9,Data Scientist,13.754647,28.252788,17.249071


In [37]:
# Plot
plt.figure(figsize=(10,7), dpi= 200)
parallel_coordinates(ml_used, 'title', colormap='tab20')
labels = ['% Theoretical Research', '% Applied Research', '% No Research']

# Lighten borders
plt.gca().spines["top"].set_alpha(0)
plt.gca().spines["bottom"].set_alpha(.3)
plt.gca().spines["right"].set_alpha(0)
plt.gca().spines["left"].set_alpha(.3)

plt.title('Use of Machine Learning in published Research by Title\n', fontsize=15, fontweight='bold', ha='center', fontname = 'arial', color = '#C41E3A')
plt.grid(alpha=0.3)
plt.xticks([0, 1, 2], labels, fontsize=9, fontweight='bold')
plt.yticks(fontsize=9, fontweight='bold')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig('./charts/Use of ML.png')
plt.close('all')

In [38]:
import plotly.graph_objects as go
import plotly.express as px
 
value_range = [0, 55]
 
dimensions = [
    dict(
        range=[0, ml_used.index.max()],
        label='<b>Title</b>',
        values=ml_used.index,
        tickvals = ml_used.index,
        ticktext = ml_used['title'],
    ),
    dict(
        range=value_range,
        label='<b>% Theoretical Research</b>',
        values=ml_used['pct_theoretical']
    ),
    dict(
        range=value_range,
        label='<b>% Applied Research</b>',
        values=ml_used['pct_applied']
    ),
    dict(
        range=value_range,
        label='<b>% Not used</b>',
        values=ml_used['pct_no']
    ),
]
 
fig = go.Figure(
    data=go.Parcoords(
        line=dict(
            color=ml_used.index,
            colorscale=px.colors.qualitative.G10,
        ),
        dimensions=dimensions,
    ),
)
 
fig.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=150,
        b=50,
        t=100,
    ),
    title='<b>Use of Machine Learning in published research by title</b>',
    title_font_size=20,
    title_x=0.5,
    title_font_family = 'arial',
    title_font_color = '#C41E3A',
)
 
fig.show()
 
fig.write_image('./charts/Use of ML in research_PL.png', scale=2)

## 3.Essential tasks and skill sets per Role

In [39]:
# Load packages
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.colors

In [40]:
df.head()

,age,gender,country,education_level,published,ml_used_theoretical,ml_used_applied,ml_used_no,code_exp,program_lang_Python,...,cloud_platform_SAPCloud,cloud_platform_VMwareCloud,cloud_platform_AlibabaCloud,cloud_platform_TencentCloud,cloud_platform_HuaweiCloud,cloud_platform_None,cloud_platform_Other,stage_index,count,edu_group
3,55-59,Man,France,Some college/university study without earning ...,NaN,NaN,NaN,NaN,10-20 years,Python,...,NaN,NaN,NaN,NaN,NaN,NaN,Other,3.0,1,Below Bachelor's Degree
7,30-34,Man,Germany,Bachelor’s degree,NaN,NaN,NaN,NaN,10-20 years,Python,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,Bachelor’s degree
8,70+,Man,Australia,Doctoral degree,Yes,NaN,"Yes, the research made use of machine learning...",NaN,20+ years,Python,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1,Above Master's Degree
10,55-59,Man,South Korea,Master’s degree,No,NaN,NaN,NaN,1-3 years,Python,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1,Master’s degree
13,40-44,Man,United States of America,Doctoral degree,Yes,"Yes, the research made advances related to som...",NaN,NaN,10-20 years,Python,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,Above Master's Degree


In [41]:
title_count = df.groupby('title').size()
title_count

title
Data Administrator           63
Data Analyst               1419
Data Architect               89
Data Engineer               316
Data Scientist             1818
Developer Advocate           53
Engineer (non-software)     426
ML/MLops Engineer           536
Manager                     798
Other                       706
Research Scientist          567
Software Engineer           904
Statistician                113
Teacher/professor           754
dtype: int64

In [42]:
python = df.groupby('title')['program_lang_Python'].count()
python

title
Data Administrator           37
Data Analyst               1054
Data Architect               77
Data Engineer               286
Data Scientist             1709
Developer Advocate           42
Engineer (non-software)     295
ML/MLops Engineer           524
Manager                     584
Other                       424
Research Scientist          476
Software Engineer           726
Statistician                 70
Teacher/professor           583
Name: program_lang_Python, dtype: int64

In [43]:
df.rename(columns = {'program_lang_C#':'program_lang_C_sharp'}, inplace = True)

In [44]:
python = df.groupby('title')['program_lang_Python'].count()
r = df.groupby('title')['program_lang_R'].count()
sql = df.groupby('title')['program_lang_SQL'].count()
c = df.groupby('title')['program_lang_C'].count()
c_thang = df.groupby('title')['program_lang_C_sharp'].count()
c_plus = df.groupby('title')['program_lang_C++'].count()
java = df.groupby('title')['program_lang_Java'].count()
javascript = df.groupby('title')['program_lang_Javascript'].count()
bash = df.groupby('title')['program_lang_Bash'].count()
php = df.groupby('title')['program_lang_PHP'].count()
matlab = df.groupby('title')['program_lang_MATLAB'].count()
julia = df.groupby('title')['program_lang_Julia'].count()
go = df.groupby('title')['program_lang_Go'].count()
other = df.groupby('title')['program_lang_Other'].count()

program_lang = pd.concat([title_count, python, r, sql, c, c_thang, c_plus, java, javascript, bash, php, matlab, julia, go, other], axis=1)
program_lang = program_lang.rename(columns={0: 'count'}).reset_index()
program_lang = (
    program_lang
    .eval('pct_python = program_lang_Python / count * 100')
    .eval('pct_r = program_lang_R / count * 100')
    .eval('pct_sql= program_lang_SQL / count * 100')
    .eval('pct_c = program_lang_C / count * 100')
    .eval('pct_c_sharp = program_lang_C_sharp / count * 100')
    .eval('pct_c_plus = `program_lang_C++` / count * 100')
    .eval('pct_java = program_lang_Java / count * 100')
    .eval('pct_javascript = program_lang_Javascript / count * 100')
    .eval('pct_bash = program_lang_Bash / count * 100')
    .eval('pct_php = program_lang_PHP / count * 100')
    .eval('pct_matlab = program_lang_MATLAB / count * 100')
    .eval('pct_julia = program_lang_Julia / count * 100')
    .eval('pct_go = program_lang_Go / count * 100')
    .eval('pct_other = program_lang_Other / count * 100')
    .drop([
        'program_lang_Python',
        'program_lang_R',
        'program_lang_SQL',
        'program_lang_C',
        'program_lang_C_sharp',
        'program_lang_C++',
        'program_lang_Java',
        'program_lang_Javascript',
        'program_lang_Bash',
        'program_lang_PHP',
        'program_lang_MATLAB',
        'program_lang_Julia',
        'program_lang_Go',
        'program_lang_Other',
        'count'
    ], axis=1)
)

program_lang

,title,pct_python,pct_r,pct_sql,pct_c,pct_c_sharp,pct_c_plus,pct_java,pct_javascript,pct_bash,pct_php,pct_matlab,pct_julia,pct_go,pct_other
0,Data Administrator,58.730159,20.634921,58.730159,7.936508,12.698413,4.761905,9.523810,17.460317,7.936508,9.523810,1.587302,0.000000,0.000000,7.936508
1,Data Analyst,74.277660,24.806202,59.619450,5.355884,3.030303,7.117689,5.849190,9.161381,4.157858,4.228330,5.144468,0.775194,0.563777,6.412967
2,Data Architect,86.516854,15.730337,74.157303,11.235955,24.719101,12.359551,31.460674,30.337079,19.101124,11.235955,4.494382,4.494382,7.865169,15.730337
3,Data Engineer,90.506329,15.822785,74.683544,12.025316,10.443038,14.873418,18.037975,19.303797,24.683544,5.696203,6.645570,1.265823,3.164557,11.075949
4,Data Scientist,94.004400,27.997800,59.515952,6.270627,4.510451,7.535754,9.625963,9.955996,13.311331,2.860286,7.370737,1.650165,0.935094,5.940594
5,Developer Advocate,79.245283,15.094340,41.509434,18.867925,16.981132,24.528302,33.962264,33.962264,15.094340,20.754717,5.660377,3.773585,3.773585,13.207547
6,Engineer (non-software),69.248826,11.971831,26.995305,7.276995,1.877934,10.798122,6.338028,7.511737,5.633803,2.582160,14.553991,1.173709,0.234742,8.450704
7,ML/MLops Engineer,97.761194,10.447761,40.671642,10.261194,6.343284,19.216418,13.992537,16.417910,26.492537,3.731343,8.582090,1.679104,3.544776,5.410448
8,Manager,73.182957,23.057644,45.363409,10.526316,7.644110,11.779449,12.781955,16.666667,9.649123,6.641604,4.761905,1.002506,1.629073,7.393484
9,Other,60.056657,16.147309,32.011331,7.082153,4.249292,7.507082,7.932011,9.915014,6.657224,4.107649,4.107649,1.133144,1.274788,7.790368


In [45]:
# -----------------------------------------#
# Fill out steps 1-4 to spec your diagram: #
# -----------------------------------------#

# Step 1. Specify >=2 categorical columns in flow order
cols = df['title']

# Step 2. Specify a column for the flow volume value
value = "Sales"
value_suffix = " USD"  # Specify (if any) a suffix for the value

# Step 3. Set the plot's title
title = "Breakdown of 2021 Sales by Region, Segment, and Product Category"

# Step 4. (Optional) Customize layout, font, and colors
width, height = 700, 500  # Set plot's width and height
fontsize = 14  # Set font size of labels
fontfamily = "Helvetica"  # Set font family of plot's text
bgcolor = "SeaShell"  # Set the plot's background color (use color name or hex code)
link_opacity = 0.3  # Set a value from 0 to 1: the lower, the more transparent the links
node_colors = px.colors.qualitative.G10  # Define a list of hex color codes for nodes

# ---------------------------------------#
# Code to create Sankey diagram begins!  #
# ---------------------------------------#

s = []  # This will hold the source nodes
t = []  # This will hold the target nodes
v = []  # This will hold the flow volumes between the source and target nodes
labels = np.unique(df[cols].values)  # Collect all the node labels

# Get all the links between two nodes in the data and their corresponding values
for c in range(len(cols) - 1):
    s.extend(df[cols[c]].tolist())
    t.extend(df[cols[c + 1]].tolist())
    v.extend(df[value].tolist())
links = pd.DataFrame({"source": s, "target": t, "value": v})  
links = links.groupby(["source", "target"], as_index=False).agg({"value": "sum"})

# Convert list of colors to RGB format to override default gray link colors
colors = [matplotlib.colors.to_rgb(i) for i in node_colors]  

# Create objects to hold node/label and link colors
label_colors, links["link_c"] = [], 0

# Loop through all the labels to specify color and to use label indices
c, max_colors = 0, len(colors)  # To loop through the colors array
for l in range(len(labels)):
    label_colors.append(colors[c])
    link_color = colors[c] + (link_opacity,)  # Make link more transparent than the node
    links.loc[links.source == labels[l], ["link_c"]] = "rgba" + str(link_color)
    links = links.replace({labels[l]: l})  # Replace node labels with the label's index
    if c == max_colors - 1:
        c = 0
    else:
        c += 1

# Convert colors into RGB string format for Plotly
label_colors = ["rgb" + str(i) for i in label_colors]

# Define a Plotly Sankey diagram
fig = go.Figure( 
    data=[
        go.Sankey(
            valuesuffix=value_suffix,
            node=dict(label=labels, color=label_colors),
            link=dict(
                source=links["source"],
                target=links["target"],
                value=links["value"],
                color=links["link_c"],
            ),
        )
    ]
)

# Customize plot based on earlier values
fig.update_layout(
    title_text=title,
    font_size=fontsize,
    font_family=fontfamily,
    width=width,
    height=height,
    paper_bgcolor=bgcolor,
    title={"y": 0.9, "x": 0.5, "xanchor": "center", "yanchor": "top"},  # Centers title
)

fig.show()

KeyError: "None of [Index(['Data Scientist', 'Software Engineer', 'Research Scientist', 'Other',\n       'Developer Advocate', 'Data Scientist', 'Data Scientist',\n       'Data Analyst', 'Data Engineer', 'Data Engineer',\n       ...\n       'Software Engineer', 'Data Scientist', 'Manager', 'ML/MLops Engineer',\n       'Teacher/professor', 'Data Architect', 'Data Scientist',\n       'Data Engineer', 'Engineer (non-software)', 'Research Scientist'],\n      dtype='object', length=8562)] are in the [columns]"